In [1]:
!nvidia-smi

Fri Sep 13 11:52:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Exercício 1: Leitura e Transferência de Dados

In [23]:
%%writefile exercicio1.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <fstream>
#include <iostream>
#include <chrono>

int main() {
    // Comece a medir o tempo
    auto start = std::chrono::steady_clock::now();

    // Passo 1: Ler os preços das ações de um arquivo para um host_vector
    thrust::host_vector<float> h_stock_prices;

    std::ifstream file("/content/stocks-google.txt"); // Caminho do arquivo no Colab
    if (file.is_open()) {
        float price;
        while (file >> price) {
            h_stock_prices.push_back(price);
        }
        file.close();
    } else {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return 1;
    }

    // Passo 2: Criar um device_vector e transferir os dados do host_vector para ele
    thrust::device_vector<float> d_stock_prices = h_stock_prices;

    // Fim da medição de tempo
    auto end = std::chrono::steady_clock::now();
    std::chrono::duration<float> duration = end - start;

    // Imprimir o tempo de alocação e cópia
    std::cout << "Tempo de alocação e cópia: " << duration.count() << " segundos." << std::endl;

    return 0;
}

Overwriting exercicio1.cu


In [24]:
!nvcc -arch=sm_70 -std=c++14 exercicio1.cu -o exercicio1

In [25]:
!./exercicio1

Tempo de alocação e cópia: 0.205575 segundos.


# Exercício 2: Cálculo de Médias e Extremos

In [26]:
%%writefile exercicio2.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <fstream>
#include <iostream>
#include <chrono>

int main() {
    thrust::host_vector<float> h_stock_prices;
    std::ifstream file("/content/stocks-google.txt");
    if (file.is_open()) {
        float price;
        while (file >> price) {
            h_stock_prices.push_back(price);
        }
        file.close();
    }
    thrust::device_vector<float> d_stock_prices = h_stock_prices;

    // Exercício 2: Cálculo de Médias e Extremos

    // Passo 1: Cálculo da média do período total (últimos 10 anos)
    float sum_total = thrust::reduce(d_stock_prices.begin(), d_stock_prices.end(), 0.0f); // Somar todos os valores
    float average_total = sum_total / d_stock_prices.size();  // Dividir pela quantidade de elementos

    // Passo 2: Cálculo da média dos últimos 365 dias
    int days_in_year = 365;
    int start_index = d_stock_prices.size() - days_in_year;  // Índice de início dos últimos 365 dias
    if (start_index < 0) start_index = 0;  // Certifique-se de que não acessamos fora do vetor, caso haja menos de 365 dias.

    float sum_last_year = thrust::reduce(d_stock_prices.begin() + start_index, d_stock_prices.end(), 0.0f); // Soma dos últimos 365 dias
    float average_last_year = sum_last_year / (d_stock_prices.end() - d_stock_prices.begin() + start_index); // Média dos últimos 365 dias

    // Passo 3: Encontrar o maior e o menor preço no período total
    auto minmax_total = thrust::minmax_element(d_stock_prices.begin(), d_stock_prices.end());
    float min_total = *minmax_total.first;  // Menor valor no período total
    float max_total = *minmax_total.second;  // Maior valor no período total

    // Passo 4: Encontrar o maior e o menor preço nos últimos 365 dias
    auto minmax_last_year = thrust::minmax_element(d_stock_prices.begin() + start_index, d_stock_prices.end());
    float min_last_year = *minmax_last_year.first;  // Menor valor nos últimos 365 dias
    float max_last_year = *minmax_last_year.second;  // Maior valor nos últimos 365 dias

    // Exibir os resultados
    std::cout << "Preço médio total: " << average_total << std::endl;
    std::cout << "Preço médio nos últimos 365 dias: " << average_last_year << std::endl;
    std::cout << "Maior preço total: " << max_total << std::endl;
    std::cout << "Menor preço total: " << min_total << std::endl;
    std::cout << "Maior preço nos últimos 365 dias: " << max_last_year << std::endl;
    std::cout << "Menor preço nos últimos 365 dias: " << min_last_year << std::endl;

    return 0;
}


Overwriting exercicio2.cu


In [27]:
!nvcc -arch=sm_70 -std=c++14 exercicio2.cu -o exercicio2

In [28]:
!./exercicio2

Preço médio total: 1580.08
Preço médio nos últimos 365 dias: 1428.02
Maior preço total: 2200
Menor preço total: 1015.24
Maior preço nos últimos 365 dias: 2200
Menor preço nos últimos 365 dias: 1060.5


# Exercício 3: Transformações entre Vetores

In [29]:
%%writefile exercicio3.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <fstream>
#include <iostream>
#include <sstream>

struct difference_functor {
    __host__ __device__
    float operator()(const float &x, const float &y) const {
        return x - y;
    }
};

int main() {
    // Passo 1: Ler os preços das ações da Apple e da Microsoft de um arquivo para dois host_vectors
    thrust::host_vector<float> h_aapl_prices;
    thrust::host_vector<float> h_msft_prices;

    std::ifstream file("/content/stocks2.txt"); // Caminho do arquivo no Colab
    if (file.is_open()) {
        std::string line;
        while (std::getline(file, line)) {
            std::stringstream ss(line);
            float aapl_price, msft_price;
            char comma;
            ss >> aapl_price >> comma >> msft_price;
            h_aapl_prices.push_back(aapl_price);
            h_msft_prices.push_back(msft_price);
        }
        file.close();
    } else {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return 1;
    }

    // Passo 2: Transferir os dados do host_vector para o device_vector
    thrust::device_vector<float> d_aapl_prices = h_aapl_prices;
    thrust::device_vector<float> d_msft_prices = h_msft_prices;

    // Passo 3: Calcular a diferença ponto a ponto entre os dois vetores
    thrust::device_vector<float> d_differences(d_aapl_prices.size());
    thrust::transform(d_aapl_prices.begin(), d_aapl_prices.end(),
                      d_msft_prices.begin(), d_differences.begin(),
                      difference_functor());

    // Passo 4: Calcular a média das diferenças
    float sum_differences = thrust::reduce(d_differences.begin(), d_differences.end(), 0.0f);
    float average_difference = sum_differences / d_differences.size();

    // Exibir a média das diferenças
    std::cout << "A média das diferenças entre os preços das ações da Apple e da Microsoft é: " << average_difference << std::endl;

    return 0;
}

Writing exercicio3.cu


In [30]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [31]:
!./exercicio3

A média das diferenças entre os preços das ações da Apple e da Microsoft é: -265.232


Isso significa que, em média, o preço das ações da Microsoft foi maior que o das ações da Apple por esse valor ao longo do período coberto pelos dados.